<a href="https://colab.research.google.com/github/Tariqameenkhan/LangChain-RAG/blob/main/RAG-Method%20-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain-pinecone langchain-google-genai


In [3]:
# import getpass
# import os
# import time

from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

# if not os.getenv("PINECONE_API_KEY"):
#     os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [11]:
import time


index_name = "langchain-rag-project2"  # Changed to lowercase

# existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# if index_name not in existing_indexes:
pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)
    # while not pc.describe_index(index_name).status["ready"]:
    #     time.sleep(1)

index = pc.Index(index_name)

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [14]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [62]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancake sand scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)


document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)


document_5 = Document(
    page_content="Wow! That was chocalate chip pancake. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)


document_10 = Document(
    page_content="I have a bad  test  chocalate chip pancakes very bad  :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_3,
    document_5,
    document_8,
    document_10,
]

In [63]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['79051c55-6da8-40c0-b240-4552a2dffadf',
 '671140ee-d661-40e8-a81c-e7113fc2fc63',
 '8e9a0ea9-ff18-448b-beee-69febb95834c',
 '595aa208-abcb-4881-b8bc-5b4801b92bb9',
 'af38d6f4-ac6f-4880-9498-cec8400286de']

In [64]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
# Interactive loop for additional user queries
while True:
    user_input = input("Ask something: ")  # Get user input inside the loop
    if user_input.lower() == "quit":
        break
    else:
        # Perform vector search for the initial query
        results = vector_store.similarity_search(user_input, k=1)

        # Pass the initial query and results to the model
        final_answer = llm.invoke(f"Answer this user query: {user_input}, here are some references to answer: {results}")


        print(f"*Initial query answer: {final_answer}  ")



